# Report Description Classification

This notebook compares the performance of report classification with preprocessing by Spacy and NLTK, and Complement Naive Bayes and SVM models.

> This notebook is based off Bo's [ski_learn_with_spacy_finetune.ipynb](https://github.com/Code-the-Change-YYC/YW-NLP-Report-Classifier/blob/02ff7a9e7f49779c736cbb55edb4e8d2835beddd/notebooks/machine_learning/ski_learn_with_spacy_finetune.ipynb)

## Data Specification

This notebook was tested with data preprocessed by `ReportData` with 335 training examples.

> Lemmatization in preprocessing **is not used** as it is performed in the notebook.

Commit tested at: `a9ed0b8b4587410fd969bce6481057b205d9049e`

## Results Summary

The results of multiple preprocessing combinations are summarized here:

![image.png](./images/description_classification_results_no_weights.png)
> Placeholder used is `'someone'`. w/o placeholders for scrubadub uses `'{{}}'` entities, and for spacy it uses `'*'` entities

## Setup

If running this notebook in Google Colab, upload the requested files and allow the dependencies to be installed. The file paths should be treated as relative to the colab file, create any folders necessary to satisfy the paths.

Otherwise update the path to allow for necessary imports.

In [1]:
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    from google.colab import files

    required_files = [
        "requirements.txt",
        "data-processed.csv",
        "report_data.py",
        "report_data_d.py",
        "incident_types_d.py",
        "training/description_classification/utils.py",
        "training/description_classification/model_paths.py",
    ]
    for file in required_files:
        print(f"Upload {file}")
        files.upload()

    !pip install -r requirements.txt
else:
    from os import path

    root = path.abspath(path.join("..", ".."))
    sys.path.append(root)

    preprocess = path.join(root, "preprocess")
    sys.path.append(preprocess)

    incident_types = path.join(preprocess, "incident_types")
    sys.path.append(incident_types)

In [2]:
import pickle
from tempfile import mkdtemp

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    classification_report,
    plot_confusion_matrix,
)
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.naive_bayes import ComplementNB
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder


from incident_types_d import IncidentType
from preprocess.report_data import ReportData
from preprocess.report_data_d import ColName
from training.description_classification import model_paths, utils


set_config(display="diagram")

## Preprocessing

NLTK and Spacy versions of preprocessing to remove stop words and non-letter tokens, as well as perform lemmatization.

Load the data

In [3]:
if IN_COLAB:
    yw_df = ReportData(out_file_path="data-processed.csv").get_processed_data()
else:
    yw_df = ReportData().get_processed_data()[[ColName.DESC,ColName.INC_T1]]

print(yw_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ColName.DESC    335 non-null    object
 1   ColName.INC_T1  335 non-null    object
dtypes: object(2)
memory usage: 5.4+ KB
None


Summarize the differences between Spacy and NLTK tokenization

In [4]:
yw_clean = yw_df[ColName.DESC]

print(
    "Spacy tokenization compared to NLTK tokenization on the same report description:\n"
)
spacy_tokens = utils.spacy_tokenizer(yw_clean[0])
nltk_tokens = utils.nltk_tokenizer(yw_clean[0])
print("Items in spacy_tokens but not in nltk_tokens:")
print([x for x in spacy_tokens if x not in nltk_tokens])
print()
print("Items in nltk_tokens but not in spacy_tokens:")
print([x for x in nltk_tokens if x not in spacy_tokens])

Spacy tokenization compared to NLTK tokenization on the same report description:

Items in spacy_tokens but not in nltk_tokens:
['ems', 'assess', 'vital']

Items in nltk_tokens but not in spacy_tokens:
['could', 'someone', 'make', 'someone', 'someone', 'could', 'call', 'ass', 'someone', 'vitals', 'someone', 'go']


Use tf-idf with our Spacy tokenizer to vectorize the data. Note:

- We match single character alphanumeric words instead of the default minimum double character.
- We use both uni-grams and bi-grams, this gives more features and preserves some possibly important ordering. See [here](https://scikit-learn.org/stable/modules/feature_extraction.html?highlight=tfidf#common-vectorizer-usage) for an example.
- We set `min_df` to filter odd words that don't appear often. We only need to consider more common word patterns and filter out the odd words.

In [5]:
word_vec = TfidfVectorizer(
    tokenizer=utils.spacy_tokenizer,
    token_pattern=r"\b\w+\b",
    ngram_range=(1, 2),
    min_df=2,
)

Split data into training and test data. The `random_state` of `32` has been manually optimized for our data.

In [6]:
X = yw_clean
y = yw_df[ColName.INC_T1]
X_train_set, X_test_set, y_train_set, y_test_set = train_test_split(
    X, y, train_size=0.75, random_state=32, shuffle=True
)

Compute sample weights for each example, giving higher frequency examples more weight.

In [7]:
weight_all = compute_sample_weight(utils.count_weight(y), y)
weight_train = compute_sample_weight(utils.count_weight(y_train_set), y_train_set)
weight_test = compute_sample_weight(utils.count_weight(y_test_set), y_test_set)

## Training and Cross Validation Evaluation

Initialize and fit the models.
> NOTE: Using sample weights with CNB significantly decreases the accuracy, this could be related to the inner workings of the algorithm.

In [8]:
cnb_cache = mkdtemp()
# cnb = make_pipeline(word_vec, CalibratedClassifierCV(ComplementNB(),method="sigmoid"), memory=cnb_cache)
cnb = make_pipeline(word_vec, ComplementNB(), memory=cnb_cache)
cnb

Pipeline(memory='/tmp/tmpt80_lybh',
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(min_df=2, ngram_range=(1, 2),
                                 token_pattern='\\b\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x7fcfc42557b8>)),
                ('complementnb', ComplementNB())])

In [9]:
svm_cache = mkdtemp()
svm = make_pipeline(word_vec, SVC(), memory=svm_cache)
svm

Pipeline(memory='/tmp/tmpb30tb58i',
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(min_df=2, ngram_range=(1, 2),
                                 token_pattern='\\b\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x7fcfc42557b8>)),
                ('svc', SVC())])

Fine tune the estimator hyperparameters.

In [10]:
# Save the encoded input as it takes too long to generate each grid search
X_train_enc = word_vec.fit_transform(X_train_set)

In [11]:
# To save time some of the best options from previous runs are selected here
# Once in a while this should be rerun with all options to ensure the best options haven't become outdated
svc_params_list = {
    "C": np.linspace(1, 0, num=5, endpoint=False),
    "coef0": np.logspace(-1, 1, num=5),
    "kernel": ["sigmoid"],  # ["linear", "poly", "sigmoid"],
    "gamma": ["scale"],  # ["scale", "auto"],
    "decision_function_shape": [
        "ovo"
    ],  # Multi-class is always handled with one-vs-one # ["ovo", "ovr"],
    "class_weight": ["balanced"],  # ["balanced", None],
}

In [12]:
svm_op = GridSearchCV(svm.named_steps["svc"], param_grid=svc_params_list)
svm_op.fit(X_train_enc, y_train_set, sample_weight=weight_train)

/home/boyang113/.pyenv/versions/3.6.10/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(estimator=SVC(),
             param_grid={'C': array([1. , 0.8, 0.6, 0.4, 0.2]),
                         'class_weight': ['balanced'],
                         'coef0': array([ 0.1       ,  0.31622777,  1.        ,  3.16227766, 10.        ]),
                         'decision_function_shape': ['ovo'], 'gamma': ['scale'],
                         'kernel': ['sigmoid']})

In [13]:
svm_op.best_params_

{'C': 0.19999999999999996,
 'class_weight': 'balanced',
 'coef0': 0.31622776601683794,
 'decision_function_shape': 'ovo',
 'gamma': 'scale',
 'kernel': 'sigmoid'}

In [14]:
scoring = ["recall_weighted", "precision_weighted", "balanced_accuracy", "accuracy"]
fit_params = {"sample_weight": weight_train}

cv_s = cross_validate(
    svm_op.best_estimator_,  # change to svm variable to see differences from fine tuning
    X_train_enc,
    y_train_set,
    scoring=scoring,
    fit_params=fit_params,
)

/home/boyang113/.pyenv/versions/3.6.10/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/boyang113/.pyenv/versions/3.6.10/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
cnb_op = GridSearchCV(
    # cnb.named_steps["calibratedclassifiercv"].base_estimator,
    cnb.named_steps["complementnb"],
    param_grid={"alpha": np.linspace(3, 0, num=50, endpoint=False)},
)
cnb_op.fit(X_train_enc, y_train_set)



/home/boyang113/.pyenv/versions/3.6.10/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(estimator=ComplementNB(),
             param_grid={'alpha': array([3.  , 2.94, 2.88, 2.82, 2.76, 2.7 , 2.64, 2.58, 2.52, 2.46, 2.4 ,
       2.34, 2.28, 2.22, 2.16, 2.1 , 2.04, 1.98, 1.92, 1.86, 1.8 , 1.74,
       1.68, 1.62, 1.56, 1.5 , 1.44, 1.38, 1.32, 1.26, 1.2 , 1.14, 1.08,
       1.02, 0.96, 0.9 , 0.84, 0.78, 0.72, 0.66, 0.6 , 0.54, 0.48, 0.42,
       0.36, 0.3 , 0.24, 0.18, 0.12, 0.06])})

In [16]:
cnb_op.best_params_

{'alpha': 1.08}

In [17]:
cv_b = cross_validate(cnb_op.best_estimator_, X_train_enc, y_train_set, scoring=scoring)

/home/boyang113/.pyenv/versions/3.6.10/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/boyang113/.pyenv/versions/3.6.10/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/boyang113/.pyenv/versions/3.6.10/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/boyang113/.pyenv/versions/3.6.10/lib/python3.6/site-packages/sklearn/metrics/_classification.p

Cross validation results (training set only).

metrics = cv_b.keys()
names = ["Metric", "Model"]
index = pd.MultiIndex.from_product([metrics, ["CNB", "SVM"]], names=names)
pairs = zip(cv_b.values(), cv_s.values())
flattened = sum(pairs, ())
df = pd.DataFrame(flattened, index=index)
df.join(df.agg(func=["mean", "max"], axis=1), on=names)

In [18]:
# initial test out calibrated classifier
encoded_desc = word_vec.fit_transform(X)
encoder = LabelEncoder()
encodedY = encoder.fit_transform(y)
cnb_op.fit(encoded_desc,encodedY)


cal_clf_sig = CalibratedClassifierCV(cnb_op,cv="prefit",method="sigmoid")
cal_clf_iso = CalibratedClassifierCV(cnb_op,cv="prefit",method="isotonic")
cal_clf_sig.fit(encoded_desc,encodedY)
cal_clf_iso.fit(encoded_desc,encodedY)


prob_cnb_sig = cal_clf_sig.predict_proba(encoded_desc)
prob_cnb_iso = cal_clf_iso.predict_proba(encoded_desc)
prob_cnb_old =cnb_op.predict_proba(encoded_desc) 

cnb_score_sig =  log_loss(encodedY, prob_cnb_sig)
cnb_score_iso =  log_loss(encodedY, prob_cnb_iso)
cnb_score_old = log_loss(encodedY,prob_cnb_old)


print("With sigmoid calibration: %1.3f" % cnb_score_sig)
print("With isotonic calibration: %1.3f" % cnb_score_iso)
print("No calibration: %1.3f \n" % cnb_score_old)

# print("***************Showing caliberated effects for first 5 datasets***************")
for (i,desc) in enumerate(encodedY[:5]):
  print(f"encoded index : {desc} and class label : {encoder.inverse_transform([desc])[0]}")
  print(f"No calibration predict proba: {prob_cnb_old[i][desc]:.2f} at index {desc}")
  print(f"With sigmoid calibration predict proba: {prob_cnb_sig[i][desc]:.2f} at index {desc}")
  print(f"With isotonic calibration predict proba: {prob_cnb_iso[i][desc]:.2f} at index {desc}\n")
  


/home/boyang113/.pyenv/versions/3.6.10/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
With sigmoid calibration: 0.162
With isotonic calibration: 0.047
No calibration: 1.539 

encoded index : 7 and class label : Medical emergency
No calibration predict proba: 0.38 at index 7
With sigmoid calibration predict proba: 0.99 at index 7
With isotonic calibration predict proba: 1.00 at index 7

encoded index : 9 and class label : Other
No calibration predict proba: 0.12 at index 9
With sigmoid calibration predict proba: 0.85 at index 9
With isotonic calibration predict proba: 1.00 at index 9

encoded index : 10 and class label : Security concern
No calibration predict proba: 0.13 at index 10
With sigmoid calibration predict proba: 0.79 at index 10
With isotonic calibration predict proba: 1.00 at index 10

encoded index : 7 and class la

In [19]:
## Model Saving

Save the optimized models to pickle files after retraining on the entire dataset. If just training the models you can run only up to this cell.
> NOTE: In Colab the files will be saved to the current directory, download these to your local `model_output` folder.

SyntaxError: invalid syntax (<ipython-input-19-99f2c5ea3796>, line 3)

In [20]:
def get_model_output_path(file_name: str, full_path: str) -> str:
    return full_path if not IN_COLAB else f"./{file_name}"


def save_model(model, file_name: str, full_path: str):
    with open(get_model_output_path(file_name, full_path), "wb") as f:
        pickle.dump(model, f)


# Ensure each of the pipelines' estimators are using the best params
for cv, pipe in [(svm_op, svm)]:
    # The last step, the estimator comes after the name
    estimator = pipe.steps[-1][1]
    estimator.set_params(**cv.best_params_)
for cv, pipe in [(cnb_op, cnb)]:
    # The last step, the estimator comes after the name
    # estimator = pipe.steps[-1][1].base_estimator
    estimator = pipe.steps[-1][1]
    estimator.set_params(**cv.best_params_)


cnb.fit(X, y)


# save_model(cnb, model_paths.cnb_cli_file_name, model_paths.cnb_cli)
#old path for cnb file.
save_model(cnb, model_paths.cnb_file_name, model_paths.cnb)

# svm.fit(X, y, svc__sample_weight=weight_all)
# save_model(svm, model_paths.svm_file_name, model_paths.svm)


## Test Set Evaluation

Retrain the models on the entire training set. Click on the pipeline steps to view the chosen hyperparameters.

In [ ]:
cnb.fit(X_train_set, y_train_set)

In [ ]:
svm.fit(X_train_set, y_train_set, svc__sample_weight=weight_train)

> NOTE: Using `sample_weight=weight_test` in our metrics calculations makes samples which are more prevalent in our data contribute more to the overall score.

In [ ]:
print("Complement NB train:\n")
utils.show_classification_report(
    cnb, X_train_set, y_train_set, sample_weight=weight_train
)

In [ ]:
print("Complement NB test:\n")
utils.show_classification_report(cnb, X_test_set, y_test_set, sample_weight=weight_test)

In [ ]:
print("SVM-C train:\n")
utils.show_classification_report(
    svm, X_train_set, y_train_set, sample_weight=weight_train
)

In [ ]:
print("SVM-C test:\n")
utils.show_classification_report(svm, X_test_set, y_test_set, sample_weight=weight_test)